# **rinna版GPT-2のテキスト生成(Ubuntu)** 

# GPUが使えるか確認

In [ ]:
import torch
print(torch.cuda.is_available())

In [ ]:
!git clone https://github.com/huggingface/transformers 

In [ ]:
!pip install transformers==4.5.0
!pip install datasets==1.2.1
!pip install sentencepiece==0.1.91

# **run_clm.pyの変更**
```
 ./transformers/examples/language-modeling/run_clm.py
 ```
## 追加
```
from transformers import T5Tokenizer
```
## 変更
```
check_min_version("4.5.0")
tokenizer = T5Tokenizer.from_pretrained(model_args.tokenizer_name, **tokenizer_kwargs)
tokenizer = T5Tokenizer.from_pretrained(model_args.model_name_or_path, **tokenizer_kwargs)
```


## **ファインチューニングの実行**
work内にtrain.txtを置く


In [ ]:
%%time

# 事前学習の実行
!python ./transformers/examples/pytorch/language-modeling/run_clm.py \
    --model_name_or_path=rinna/japanese-gpt2-medium \
    --train_file=train.txt \
    --validation_file=train.txt \
    --do_train \
    --do_eval \
    --num_train_epochs=3 \
    --save_steps=5000 \
    --save_total_limit=3 \
    --per_device_train_batch_size=1 \
    --per_device_eval_batch_size=1 \
    --output_dir=output3/ \
    --use_fast_tokenizer=False

In [ ]:
from transformers import AutoModelForCausalLM
from transformers import T5Tokenizer
import csv

#入力文字
enter = ['今日はお腹空いたので', '明日の晩御飯は']
name = 'hariji'
num = 1 

header = ["入力","出力"]
with open("{0}_gen.csv".format(name), 'w', encoding="utf-8") as f:
  writer = csv.writer(f, lineterminator="\n")
  writer.writerow(header)

# トークナイザーとモデルの準備
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")
model = AutoModelForCausalLM.from_pretrained("output3/")

for sentence in enter:
  print(num)
  # 推論
  input = tokenizer.encode(sentence, return_tensors="pt")
  output = model.generate(input, do_sample=True, max_length=100, num_return_sequences=25)
  outer = tokenizer.batch_decode(output)

  #</s>の削除
  outer = [s.replace('</s> ','') for s in outer]
  length = len(outer)
  # for i in range(length):
  #   output[i] = output[i].replace('</s> ','')
  #   print(output[i])

  #CSV書き込み
  header = ["入力","出力"]
  with open("{0}_gen.csv".format(name), 'a', encoding="utf-8") as f:
    writer = csv.writer(f, lineterminator="\n")
    one_row = [sentence, '']
    writer.writerow(one_row)
    for i in range(length):
      one_row = ["", outer[i]]
      writer.writerow(one_row)
  num += 1

print('complete')


CSV書き込み作成

In [ ]:
import csv
header = ["入力","出力"]
with open("output.csv", 'w', encoding="sjis") as f:
  writer = csv.writer(f, lineterminator="\n")
  length = len(outer)
  for i in range(length):
    print(outer[i])
    
    one_row = ["", outer[i]]
    writer.writerow(one_row)